<a href="https://colab.research.google.com/github/MariiaZimokha/notebooks/blob/main/kaggle/Tabular_Regression_with_the_California_Housing_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set up

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
file_path = "/content/drive/MyDrive/Studying/kaggle.json"

In [3]:
! mkdir ~/.kaggle
! cp /content/drive/MyDrive/Studying/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle competitions download -c playground-series-s3e1

100% 2.33M/2.33M [00:00<00:00, 4.78MB/s]
100% 2.33M/2.33M [00:00<00:00, 4.13MB/s]


In [ ]:
! pip install -q kaggle
! pip install xgboost==1.5.1

## Imports

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from zipfile import ZipFile

import os.path

## Unzip data

In [7]:
base_path = "/content/"
file_name = "playground-series-s3e1.zip"

with ZipFile(f"{base_path}{file_name}", 'r') as zip_ref:
    zip_ref.extractall(base_path)


## Read data

In [8]:
train_df = pd.read_csv("/content/train.csv")
test_df = pd.read_csv("/content/test.csv")

## Data exploration 

In [9]:
train_df.head()

,id,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,0,2.3859,15.0,3.827160,1.112100,1280.0,2.486989,34.60,-120.12,0.980
1,1,3.7188,17.0,6.013373,1.054217,1504.0,3.813084,38.69,-121.22,0.946
2,2,4.7750,27.0,6.535604,1.103175,1061.0,2.464602,34.71,-120.45,1.576
3,3,2.4138,16.0,3.350203,0.965432,1255.0,2.089286,32.66,-117.09,1.336
4,4,3.7500,52.0,4.284404,1.069246,1793.0,1.604790,37.80,-122.41,4.500


In [11]:
train_df.describe()

,id,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
count,37137.00000,37137.000000,37137.000000,37137.000000,37137.000000,37137.000000,37137.000000,37137.000000,37137.000000,37137.000000
mean,18568.00000,3.851029,26.057005,5.163124,1.062204,1660.778919,2.831243,35.570030,-119.554329,2.079751
std,10720.67281,1.803167,12.158221,1.206242,0.096490,1302.469608,2.702413,2.083179,1.974028,1.158571
min,0.00000,0.499900,2.000000,0.851064,0.500000,3.000000,0.950000,32.550000,-124.350000,0.149990
25%,9284.00000,2.602300,17.000000,4.357522,1.020305,952.000000,2.394495,33.930000,-121.800000,1.208000
50%,18568.00000,3.515600,25.000000,5.068611,1.054545,1383.000000,2.744828,34.190000,-118.450000,1.808000
75%,27852.00000,4.699700,35.000000,5.858597,1.088825,1856.000000,3.125313,37.700000,-118.020000,2.660000
max,37136.00000,15.000100,52.000000,28.837607,5.873181,35682.000000,502.990610,41.950000,-114.550000,5.000010


In [12]:
train_df.info

<bound method DataFrame.info of           id  MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  \
0          0  2.3859      15.0  3.827160   1.112100      1280.0  2.486989   
1          1  3.7188      17.0  6.013373   1.054217      1504.0  3.813084   
2          2  4.7750      27.0  6.535604   1.103175      1061.0  2.464602   
3          3  2.4138      16.0  3.350203   0.965432      1255.0  2.089286   
4          4  3.7500      52.0  4.284404   1.069246      1793.0  1.604790   
...      ...     ...       ...       ...        ...         ...       ...   
37132  37132  3.3438      50.0  4.936508   1.079365      1775.0  3.022222   
37133  37133  3.7308      26.0  5.087533   0.966019      1006.0  4.316901   
37134  37134  4.1716      52.0  4.678862   1.101485      1156.0  1.431734   
37135  37135  2.7143      16.0  5.710074   1.068376       584.0  2.803659   
37136  37136  2.2419      34.0  5.424419   1.058685      1340.0  3.799065   

       Latitude  Longitude  MedHouseVal  
0

## xgboost Model

In [14]:
import xgboost
print(xgboost.__version__)

1.5.1


In [50]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

def model_train(X, y, n_estimators=15, max_depth=20, eval_set=[], eval_metric=[]):
  xgb_model = xgboost.XGBRegressor(
      n_estimators=n_estimators, 
      max_depth=max_depth, 
      objective='reg:squarederror',
      colsample_bytree=0.7, 
      learning_rate=0.1,
      verbosity=2)

  # xgb_model.fit(X, y)
  xgb_model.fit(X, y)
  
  return xgb_model
  

def get_metrics(xgb_model, X_test, y_test):
  ypred = xgb_model.predict(X_test)
  mse = mean_squared_error(y_test, ypred)
  print(f"MSE: {mse}")

  print(f"RMSE: {(mse**(1/2.0))}")


def get_predictions(model, test_df=test_df, features=[]):
  test_x = test_df.loc[:, features]
  y_test_pred = model.predict(test_x)
  return pd.DataFrame(data={'id': test_df.id, 'MedHouseVal': y_test_pred})

###Training

In [32]:
# features = ["MedInc",  "HouseAge",  "AveRooms",  "AveBedrms",  "Population",  "AveOccup", "Latitude",  "Longitude"]
features = ["MedInc", "HouseAge",  "AveOccup", "Latitude",  "Longitude"]
train_y = train_df["MedHouseVal"]
train_x = train_df.loc[:, features]

X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.1, random_state=42)


eval_set = [(X_train, y_train), (X_test, y_test)]
eval_metric = ["auc","error"]

# colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 30
xgb_model = model_train(X_train, y_train, n_estimators=30, max_depth=10)
print('Train metrics')
get_metrics(xgb_model, X_train, y_train)
print('Test metrics')
get_metrics(xgb_model, X_test, y_test)

[10:32:26] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 906 extra nodes, 0 pruned nodes, max_depth=10
[10:32:26] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 914 extra nodes, 0 pruned nodes, max_depth=10
[10:32:26] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 982 extra nodes, 0 pruned nodes, max_depth=10
[10:32:26] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 1078 extra nodes, 0 pruned nodes, max_depth=10
[10:32:26] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 1132 extra nodes, 0 pruned nodes, max_depth=10
[10:32:26] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 1148 extra nodes, 0 pruned nodes, max_depth=10
[10:32:26] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 1324 extra nodes, 0 pruned nodes, max_depth=10
[10:32:26] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 1260 extra nodes, 0 pruned nodes, max_depth=10
[10:32:26] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 1086 e

In [ ]:
# Train metrics
# MSE: 0.20761675334790927
# RMSE: 0.4556498143837099
# Test metrics
# MSE: 0.35886881510238194
# RMSE: 0.5990566042557096

In [51]:
submission = get_predictions(xgb_model, test_df, features)

In [52]:
submission.head()
# 0	37137	0.743201
# 1	37138	1.310513
# 2	37139	4.044755
# 3	37140	2.906413
# 4	37141	2.207206


# 0	37137	0.643827
# 1	37138	1.093985
# 2	37139	3.553613
# 3	37140	3.140647
# 4	37141	2.226977

,id,MedHouseVal
0,37137,0.664268
1,37138,1.269625
2,37139,4.012249
3,37140,3.283367
4,37141,2.336932


### Fine tune


In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

# features = ["MedInc",  "HouseAge",  "AveRooms",  "AveBedrms",  "Population",  "AveOccup", "Latitude",  "Longitude"]
features = ["MedInc",  "AveOccup", "Latitude",  "Longitude"]
train_y = train_df["MedHouseVal"]
train_x = train_df.loc[:, features]

X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.33, random_state=42)

params = { 
    'max_depth': [3, 6, 10, 15, 17, 20, 25],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [5, 7, 10, 15, 20, 25, 30],
    'colsample_bytree': [0.3, 0.7]
}

xgbr = xgb.XGBRegressor(seed = 20)

clf = GridSearchCV(estimator=xgbr, 
                   param_grid=params,
                   scoring='neg_mean_squared_error', 
                   verbose=2)

clf.fit(X_train, y_train)

print("Best parameters:", clf.best_params_)
print("Lowest RMSE: ", (-clf.best_score_)**(1/2.0))

Streaming output truncated to the last 5000 lines.
[10:22:48] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:22:48] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:22:48] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:22:48] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:22:48] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:22:48] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:22:48] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:22:48] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:22:48] INFO: ../src/tree/updater_prune.cc

In [27]:
print("Best parameters:", clf.best_params_)
print("Lowest RMSE: ", (-clf.best_score_)**(1/2.0))

Best parameters: {'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 30}
Lowest RMSE:  0.6155105589203939


### push submission


In [19]:
submission.to_csv("/content/xgboost-submission.csv", index=False)

In [20]:
!kaggle competitions submit -c playground-series-s3e1 -f "/content/xgboost-submission.csv" -m "xgboost with less features"

100% 384k/384k [00:01<00:00, 380kB/s]
Successfully submitted to Playground Series Season 3, Episode 1

## Catboost model

In [ ]:
!pip install catboost

In [143]:
from catboost import Pool, CatBoostRegressor
from sklearn.model_selection import train_test_split

features = ["MedInc",  "HouseAge",  "AveRooms",  "AveBedrms",  "Population",  "AveOccup", "Latitude",  "Longitude"]
train_y = train_df["MedHouseVal"]
train_x = train_df.loc[:, features]

X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.33, random_state=42)

train_pool = Pool(X_train, y_train)
test_pool = Pool(X_test, y_test)

# {'depth': 10, 'iterations': 35, 'learning_rate': 0.1}
model = CatBoostRegressor(iterations=35, 
                          depth=10, 
                          learning_rate=0.1,
                          loss_function='RMSE')

model.fit(train_pool, eval_set=(X_test, y_test))

0:	learn: 1.0882377	test: 1.0922675	best: 1.0922675 (0)	total: 28.9ms	remaining: 983ms
1:	learn: 1.0261922	test: 1.0315128	best: 1.0315128 (1)	total: 59.1ms	remaining: 975ms
2:	learn: 0.9723239	test: 0.9790158	best: 0.9790158 (2)	total: 88.4ms	remaining: 943ms
3:	learn: 0.9276371	test: 0.9353336	best: 0.9353336 (3)	total: 118ms	remaining: 913ms
4:	learn: 0.8882484	test: 0.8975245	best: 0.8975245 (4)	total: 151ms	remaining: 904ms
5:	learn: 0.8534146	test: 0.8646260	best: 0.8646260 (5)	total: 181ms	remaining: 872ms
6:	learn: 0.8211128	test: 0.8333140	best: 0.8333140 (6)	total: 210ms	remaining: 839ms
7:	learn: 0.7942006	test: 0.8079155	best: 0.8079155 (7)	total: 243ms	remaining: 822ms
8:	learn: 0.7717894	test: 0.7863301	best: 0.7863301 (8)	total: 280ms	remaining: 810ms
9:	learn: 0.7488335	test: 0.7646530	best: 0.7646530 (9)	total: 309ms	remaining: 773ms
10:	learn: 0.7279788	test: 0.7450124	best: 0.7450124 (10)	total: 350ms	remaining: 763ms
11:	learn: 0.7129700	test: 0.7309123	best: 0.7309

In [144]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score


def metrics(X, y):
  pred = model.predict(X)
  rmse = (np.sqrt(mean_squared_error(y, pred)))
  r2 = r2_score(y, pred)
  print("performance")
  print(f"RMSE: {rmse}")
  print(f"R2: {r2}")
  # print(f"Accuracy: {accuracy_score(y, pred)}")


metrics(X_train, y_train)
metrics(X_test, y_test)


performance
RMSE: 0.589011578192903
R2: 0.7410671752610525
performance
RMSE: 0.6184756954448359
R2: 0.7160152770206737


### Submission

In [147]:
submission = get_predictions(model, test_df=test_df)

submission.to_csv("/content/catboost-submission.csv", index=False)

In [148]:
submission.head()

,id,MedHouseVal
0,37137,0.778283
1,37138,1.358030
2,37139,4.119270
3,37140,3.662458
4,37141,2.552395


###Fine tune

In [137]:
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostRegressor

model_CBR = CatBoostRegressor()

parameters = {
    'depth': [6, 8, 10, 15],
    'learning_rate' : [0.01, 0.05, 0.1, 0.01],
    'iterations'    : [10, 15, 20, 25, 30, 35]
}

grid = GridSearchCV(estimator=model_CBR, param_grid = parameters, cv = 2, n_jobs=-1)
grid.fit(X_train, y_train)

0:	learn: 1.0882377	total: 29.1ms	remaining: 989ms
1:	learn: 1.0261922	total: 56.1ms	remaining: 926ms
2:	learn: 0.9723239	total: 83.9ms	remaining: 895ms
3:	learn: 0.9276371	total: 112ms	remaining: 866ms
4:	learn: 0.8882484	total: 139ms	remaining: 835ms
5:	learn: 0.8534146	total: 167ms	remaining: 806ms
6:	learn: 0.8211128	total: 194ms	remaining: 777ms
7:	learn: 0.7942006	total: 222ms	remaining: 750ms
8:	learn: 0.7717894	total: 269ms	remaining: 778ms
9:	learn: 0.7488335	total: 301ms	remaining: 753ms
10:	learn: 0.7279788	total: 329ms	remaining: 719ms
11:	learn: 0.7129700	total: 357ms	remaining: 684ms
12:	learn: 0.6962640	total: 386ms	remaining: 652ms
13:	learn: 0.6848626	total: 416ms	remaining: 624ms
14:	learn: 0.6746093	total: 444ms	remaining: 593ms
15:	learn: 0.6649493	total: 477ms	remaining: 566ms
16:	learn: 0.6558670	total: 507ms	remaining: 537ms
17:	learn: 0.6494288	total: 545ms	remaining: 514ms
18:	learn: 0.6420156	total: 580ms	remaining: 488ms
19:	learn: 0.6360845	total: 608ms	rema

GridSearchCV(cv=2,
             estimator=<catboost.core.CatBoostRegressor object at 0x7f11f1a18220>,
             n_jobs=-1,
             param_grid={'depth': [6, 8, 10, 15],
                         'iterations': [10, 15, 20, 25, 30, 35],
                         'learning_rate': [0.01, 0.05, 0.1, 0.01]})

In [139]:
print("grid.best_estimator_", grid.best_estimator_)
print("grid.best_score_", grid.best_score_)
print("grid.best_params_", grid.best_params_)


grid.best_estimator_ <catboost.core.CatBoostRegressor object at 0x7f11f1bc2d90>
grid.best_score_ 0.7213214447992868
grid.best_params_ {'depth': 10, 'iterations': 35, 'learning_rate': 0.1}


## LGBMRegressor Model

LGBMRegressor yeild the best result for tabular data

In [ ]:
!pip install lightgbm

### Training

#### Init training

Initial train to invistigate what features are important 

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# features = ["MedInc",  "HouseAge",  "AveRooms",  "AveBedrms",  "Population",  "AveOccup", "Latitude",  "Longitude"]
features = ["MedInc", "HouseAge", "AveOccup", "Latitude",  "Longitude"]
train_y = train_df["MedHouseVal"]
train_x = train_df.loc[:, features]

X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.15, random_state=42)

# laoding data
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)


# defining parameters 
params = {
		"n_estimators": 10000,
		"learning_rate": 0.2741569247352038,
		"num_leaves": 1605,
		"max_depth": 8,
		"min_data_in_leaf": 900,
		"max_bin": 243,
		"lambda_l1": 50,
		"lambda_l2": 50,
		"min_gain_to_split": 1.6266032818136957,
		"bagging_fraction": 0.8,
		"bagging_freq": 1,
		"feature_fraction": 0.6000000000000001
}
# params = {
#     'task': 'train', 
#     'boosting': 'gbdt',
#     'objective': 'regression',
#     'num_leaves': 20,
#     'learnnig_rage': 0.1,
#     'metric': {'l2','l1'},
#     'verbose': -1
# }

reg = lgb.LGBMRegressor(verbose=-1)

lgb_model = reg.fit(
    X_train, 
    y_train,
    eval_set=[(X_test, y_test)],
    early_stopping_rounds=100
)

# prediction
y_pred = lgb_model.predict(X_train)

# accuracy check
mse = mean_squared_error(y_train, y_pred)
rmse = mse**(0.5)
print("MSE: %.2f" % mse)
print("RMSE: %.2f" % rmse) 

# prediction
y_pred = lgb_model.predict(X_test)

# accuracy check
mse = mean_squared_error(y_test, y_pred)
rmse = mse**(0.5)
print("MSE: %.2f" % mse)
print("RMSE: %.2f" % rmse) 

In [ ]:
# MSE: 0.26
# RMSE: 0.51
# MSE: 0.35
# RMSE: 0.59

# MSE: 0.29
# RMSE: 0.54
# MSE: 0.36
# RMSE: 0.60

# MSE: 0.32
# RMSE: 0.57
# MSE: 0.37
# RMSE: 0.61

After initial run with all features, I can look at importance of each feature. After I can skip adding three unimportant features to the training. It improves model performance

In [ ]:
# plotting feature importance
lgb.plot_importance(lgb_model, height=.5) 

#### Training

In [114]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold

# features = ["MedInc",  "HouseAge",  "AveRooms",  "AveBedrms",  "Population",  "AveOccup", "Latitude",  "Longitude"]
features = ["MedInc", "HouseAge", "AveOccup", "Latitude",  "Longitude"]
target = 'MedHouseVal'

train_y = train_df["MedHouseVal"]
train_x = train_df.loc[:, features]

X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.15, random_state=42)

# n_estimators: 10000
# 		learning_rate: 0.12609202228452984
# 		num_leaves: 245
# 		max_depth: 12
# 		min_data_in_leaf: 200
# 		max_bin: 290
# 		lambda_l1: 5
# 		lambda_l2: 10
# 		min_gain_to_split: 1.7268462706693724
# 		bagging_fraction: 0.8
# 		bagging_freq: 1
# 		feature_fraction: 0.6000000000000001
param_grid = {
		"n_estimators": 10000,
		"learning_rate": 0.12609202228452984,
		"num_leaves": 245,
		"max_depth": 12,
		"min_data_in_leaf": 200,
		"max_bin": 290,
		"lambda_l1": 5,
		"lambda_l2": 10,
		"min_gain_to_split": 1.7,
		"bagging_fraction": 0.8,
		"bagging_freq": 1,
		"feature_fraction": 0.6
}

# cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1121218)
kf = KFold(n_splits=7, random_state=42, shuffle=True)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

clfs = []

clf = lgb.LGBMRegressor()


for i, (train_index, val_index) in enumerate(kf.split(train_df)):
      X_train, X_val = train_df[features].loc[train_index], train_df[features].loc[val_index]
      y_train, y_val = train_df[target][train_index], train_df[target][val_index]
      

      clf.fit(X_train.values, y_train)
      preds = clf.predict(X_val.values)
    
      clfs.append(clf)
      print(f'RMSE on fold #{i}: {mean_squared_error(y_val, preds, squared=False)}')
      # cv_scores.append( mean_squared_error(y_val, preds, squared=False))

# without params
# RMSE on fold #0: 0.6006043984153631
# RMSE on fold #1: 0.5666960470723027
# RMSE on fold #2: 0.5770339229531426
# RMSE on fold #3: 0.5617203472488859
# RMSE on fold #4: 0.5614925585171172
# ==============================================================================
# prediction
y_pred = clf.predict(X_train)

# accuracy check
mse = mean_squared_error(y_train, y_pred)
rmse = mse**(0.5)
print("MSE: %.2f" % mse)
print("RMSE: %.2f" % rmse) 

# prediction
y_pred = clf.predict(X_test)

# accuracy check
mse = mean_squared_error(y_test, y_pred)
rmse = mse**(0.5)
print("MSE: %.2f" % mse)
print("RMSE: %.2f" % rmse) 

RMSE on fold #0: 0.6000204899122741
RMSE on fold #1: 0.569925017145236
RMSE on fold #2: 0.5637947258953552
RMSE on fold #3: 0.5697859265994824
RMSE on fold #4: 0.56138013614185
RMSE on fold #5: 0.5766851308041223
RMSE on fold #6: 0.5457035545834316
MSE: 0.29
RMSE: 0.54
MSE: 0.31
RMSE: 0.56


In [116]:
features = ["MedInc", "HouseAge", "AveOccup", "Latitude",  "Longitude"]
submission = get_predictions(clf, test_df, features)
submission.head()

# the best
# 0	37137	0.705786
# 1	37138	1.119464
# 2	37139	4.158363
# 3	37140	3.368294
# 4	37141	2.440503


,id,MedHouseVal
0,37137,0.699946
1,37138,1.151113
2,37139,4.064338
3,37140,3.415378
4,37141,2.482294


### Submission

In [95]:
submission.to_csv("/content/LGBMRegressor-submission-1.csv", index=False)

In [96]:
!kaggle competitions submit -c playground-series-s3e1 -f "/content/LGBMRegressor-submission-1.csv" -m "LGBMRegressor without HouseAge"

100% 596k/596k [00:01<00:00, 559kB/s]
400 - Bad Request


### Fine tune

In [ ]:
!pip install optuna

In [111]:
from matplotlib import compare_versions
import optuna 
from optuna.integration import LightGBMPruningCallback

from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, KFold

def objective(trial, X, features):
    param_grid = {
        "n_estimators": trial.suggest_categorical("n_estimators", [10000]),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 5, 3000, step=20),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
        "max_bin": trial.suggest_int("max_bin", 200, 300),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        "bagging_fraction": trial.suggest_float(
            "bagging_fraction", 0.2, 0.95, step=0.1
        ),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        "feature_fraction": trial.suggest_float(
            "feature_fraction", 0.2, 0.95, step=0.1
        ),
    }

    # cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1121218)
    kf = KFold(n_splits=5, random_state=42, shuffle=True)

    cv_scores = []
    clfs = []

    target = 'MedHouseVal'

    for i, (train_index, val_index) in enumerate(kf.split(X)):
      X_train, X_val = X[features].loc[train_index], X[features].loc[val_index]
      y_train, y_val = X[target][train_index], X[target][val_index]
      
      clf = lgb.LGBMRegressor(**param_grid)
      clf.fit(X_train.values, y_train)
      preds = clf.predict(X_val.values)
    
      clfs.append(clf)
      print(f'RMSE on fold #{i}: {mean_squared_error(y_val, preds, squared=False)}')
      cv_scores.append( mean_squared_error(y_val, preds, squared=False))

    return np.mean(cv_scores)

features = ["MedInc", "HouseAge", "AveOccup", "Latitude",  "Longitude"]
train_y = train_df["MedHouseVal"]

study = optuna.create_study(direction="minimize", study_name="LGBM Regressor")
func = lambda trial: objective(trial, train_df, features)
study.optimize(func, n_trials=150)

[I 2023-01-05 14:09:29,193] A new study created in memory with name: LGBM Regressor
/usr/local/lib/python3.8/dist-packages/optuna/distributions.py:683: UserWarning: The distribution is specified by [5, 3000] and step=20, but the range is not divisible by `step`. It will be replaced by [5, 2985].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/optuna/distributions.py:668: UserWarning: The distribution is specified by [0.2, 0.95] and step=0.1, but the range is not divisible by `step`. It will be replaced by [0.2, 0.9].
  warnings.warn(


RMSE on fold #0: 0.8006232707877727
RMSE on fold #1: 0.7687570244340598
RMSE on fold #2: 0.7848469910389523
RMSE on fold #3: 0.7737991405955882


[I 2023-01-05 14:09:43,710] Trial 0 finished with value: 0.7806970364723753 and parameters: {'n_estimators': 10000, 'learning_rate': 0.04009234014187899, 'num_leaves': 105, 'max_depth': 4, 'min_data_in_leaf': 7800, 'max_bin': 262, 'lambda_l1': 5, 'lambda_l2': 100, 'min_gain_to_split': 13.41435582346971, 'bagging_fraction': 0.9, 'bagging_freq': 1, 'feature_fraction': 0.2}. Best is trial 0 with value: 0.7806970364723753.


RMSE on fold #4: 0.7754587555055037


/usr/local/lib/python3.8/dist-packages/optuna/distributions.py:683: UserWarning: The distribution is specified by [5, 3000] and step=20, but the range is not divisible by `step`. It will be replaced by [5, 2985].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/optuna/distributions.py:668: UserWarning: The distribution is specified by [0.2, 0.95] and step=0.1, but the range is not divisible by `step`. It will be replaced by [0.2, 0.9].
  warnings.warn(


...

RMSE on fold #4: 0.5777367190431596


/usr/local/lib/python3.8/dist-packages/optuna/distributions.py:683: UserWarning: The distribution is specified by [5, 3000] and step=20, but the range is not divisible by `step`. It will be replaced by [5, 2985].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/optuna/distributions.py:668: UserWarning: The distribution is specified by [0.2, 0.95] and step=0.1, but the range is not divisible by `step`. It will be replaced by [0.2, 0.9].
  warnings.warn(


RMSE on fold #0: 0.6199309750730714
RMSE on fold #1: 0.5865164871559376
RMSE on fold #2: 0.5958341883968576
RMSE on fold #3: 0.5888157465864883


[I 2023-01-05 15:15:52,877] Trial 149 finished with value: 0.5945761511075875 and parameters: {'n_estimators': 10000, 'learning_rate': 0.24311322839692442, 'num_leaves': 1405, 'max_depth': 8, 'min_data_in_leaf': 500, 'max_bin': 279, 'lambda_l1': 5, 'lambda_l2': 25, 'min_gain_to_split': 1.2796127690556784, 'bagging_fraction': 0.6000000000000001, 'bagging_freq': 1, 'feature_fraction': 0.4}. Best is trial 32 with value: 0.5724572354882351.


RMSE on fold #4: 0.5817833583255829


In [112]:
print(f"\tBest value (rmse): {study.best_value:.5f}")
print(f"\tBest params:")

for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")

	Best value (rmse): 0.57246
	Best params:
		n_estimators: 10000
		learning_rate: 0.12609202228452984
		num_leaves: 245
		max_depth: 12
		min_data_in_leaf: 200
		max_bin: 290
		lambda_l1: 5
		lambda_l2: 10
		min_gain_to_split: 1.7268462706693724
		bagging_fraction: 0.8
		bagging_freq: 1
		feature_fraction: 0.6000000000000001


## Feature scaling

In [64]:
train_df.describe()

,id,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
count,37137.00000,37137.000000,37137.000000,37137.000000,37137.000000,37137.000000,37137.000000,37137.000000,37137.000000,37137.000000
mean,18568.00000,3.851029,26.057005,5.163124,1.062204,1660.778919,2.831243,35.570030,-119.554329,2.079751
std,10720.67281,1.803167,12.158221,1.206242,0.096490,1302.469608,2.702413,2.083179,1.974028,1.158571
min,0.00000,0.499900,2.000000,0.851064,0.500000,3.000000,0.950000,32.550000,-124.350000,0.149990
25%,9284.00000,2.602300,17.000000,4.357522,1.020305,952.000000,2.394495,33.930000,-121.800000,1.208000
50%,18568.00000,3.515600,25.000000,5.068611,1.054545,1383.000000,2.744828,34.190000,-118.450000,1.808000
75%,27852.00000,4.699700,35.000000,5.858597,1.088825,1856.000000,3.125313,37.700000,-118.020000,2.660000
max,37136.00000,15.000100,52.000000,28.837607,5.873181,35682.000000,502.990610,41.950000,-114.550000,5.000010


In [65]:
test_df.describe()

,id,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
count,24759.000000,24759.000000,24759.000000,24759.000000,24759.000000,24759.000000,24759.000000,24759.000000,24759.000000
mean,49516.000000,3.832618,26.041561,5.168789,1.063599,1679.327548,2.830808,35.598007,-119.570208
std,7147.451994,1.797503,12.177907,1.252874,0.123630,1365.598976,1.615320,2.097787,1.977200
min,37137.000000,0.499900,2.000000,1.000000,0.560000,3.000000,0.764706,32.550000,-124.230000
25%,43326.500000,2.590150,17.000000,4.356443,1.020460,955.000000,2.400000,33.930000,-121.800000
50%,49516.000000,3.504600,25.000000,5.077143,1.054094,1398.000000,2.751592,34.200000,-118.460000
75%,55705.500000,4.687500,35.000000,5.858646,1.088295,1874.000000,3.129167,37.720000,-118.020000
max,61895.000000,15.000100,52.000000,56.269231,10.500000,35682.000000,230.172414,41.950000,-114.550000


In [69]:
train_df.columns

Index(['id', 'MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population',
       'AveOccup', 'Latitude', 'Longitude', 'MedHouseVal'],
      dtype='object')

In [97]:
def scaling(df):
  new_df = df.copy()
  # new_df.drop(["id"], axis=1, inplace=True)
  
  columns = new_df.columns

  for col in columns:
    if col != "id":
      max = df[col].max()
      new_df[col] /= max

  return new_df

In [98]:
df_scaled_train = scaling(train_df)

In [99]:
df_scaled_train.head()

,id,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,0,0.159059,0.288462,0.132714,0.189352,0.035872,0.004944,0.824791,1.048625,0.196000
1,1,0.247918,0.326923,0.208525,0.179497,0.042150,0.007581,0.922288,1.058228,0.189200
2,2,0.318331,0.519231,0.226635,0.187833,0.029735,0.004900,0.827414,1.051506,0.315199
3,3,0.160919,0.307692,0.116175,0.164380,0.035172,0.004154,0.778546,1.022174,0.267199
4,4,0.249998,1.000000,0.148570,0.182056,0.050249,0.003190,0.901073,1.068616,0.899998


In [100]:
features = ["MedInc",  "HouseAge",  "AveRooms",  "AveBedrms",  "Population",  "AveOccup", "Latitude",  "Longitude"]
y = df_scaled_train["MedHouseVal"]
X = df_scaled_train.loc[:, features]

xgb_model = model_train(X[:30000], y[:30000], n_estimators=5, max_depth=15)
print('Train metrics')
get_metrics(xgb_model, X[:30000], y[:30000])
print('Test metrics')
get_metrics(xgb_model, X[30000:], y[30000:])



[14:57:33] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 6554 extra nodes, 0 pruned nodes, max_depth=15
[14:57:33] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 8106 extra nodes, 0 pruned nodes, max_depth=15
[14:57:33] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 8520 extra nodes, 0 pruned nodes, max_depth=15
[14:57:34] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 8402 extra nodes, 0 pruned nodes, max_depth=15
[14:57:34] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 8862 extra nodes, 0 pruned nodes, max_depth=15
Train metrics
MSE: 0.0051278816282317105
RMSE: 0.07160922865267934
Test metrics
MSE: 0.018056726548610253
RMSE: 0.13437531971537875


In [101]:
df_scaled_test = scaling(test_df)
submission = get_predictions(xgb_model, df_scaled_test)

In [102]:
submission.head()

,id,MedHouseVal
0,37137,0.390057
1,37138,0.375561
2,37139,0.527263
3,37140,0.550660
4,37141,0.403925


In [103]:
submission.to_csv("/content/scailed-xgboost-submission.csv", index=False)

In [104]:
!kaggle competitions submit -c playground-series-s3e1 -f "/content/scailed-xgboost-submission.csv" -m "scailed"

100% 402k/402k [00:00<00:00, 1.97MB/s]
Successfully submitted to Playground Series Season 3, Episode 1